# Build some workbooks with requisite data for modelling industry

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from openpyxl import Workbook
import xlsxwriter
import pandas.io.formats.excel
import glob
from pandas import ExcelWriter

In [2]:
# Load data frames

EGEDA_years = pd.read_csv('../data/input_data/EGEDA_to2017.csv')
historical_production = pd.read_csv('../data/input_data/historical_production.csv')
GDP = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'GDP')
GDP_growth = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'GDP_growth')
population = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'Population')
GDP_per_capita = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'GDP_per_capita')
GDP_per_capita_percent = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'GDP_per_capita_%')
sectoral_GDP = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'Sectoral_GDP')
sectoral_GDP_percent = pd.read_excel('../data/input_data/Macro.xlsx', sheet_name = 'Sectoral_GDP_%')

# 7th production data (historical through to 2050)
steel_7th = pd.read_excel('../data/input_data/7th_prod.xlsx', sheet_name = 'Steel', skiprows = 2)
aluminium_7th = pd.read_excel('../data/input_data/7th_prod.xlsx', sheet_name = 'Aluminium', skiprows = 2)
cement_7th = pd.read_excel('../data/input_data/7th_prod.xlsx', sheet_name = 'Cement', skiprows = 2)
chemicals_7th = pd.read_excel('../data/input_data/7th_prod.xlsx', sheet_name = 'Chemicals', skiprows = 2)
mining_7th = pd.read_excel('../data/input_data/7th_prod.xlsx', sheet_name = 'Mining', skiprows = 2)
pulp_paper_7th = pd.read_excel('../data/input_data/7th_prod.xlsx', sheet_name = 'Pulp_paper', skiprows = 2)
non_specified_7th = pd.read_excel('../data/input_data/7th_prod.xlsx', sheet_name = 'Non_specified', skiprows = 2)
other_7th = pd.read_excel('../data/input_data/7th_prod.xlsx', sheet_name = 'Other', skiprows = 2)

# Change numeric year column names to string (so that it appends to histroical_production later in the code; which has string year column names)
steel_7th.columns = steel_7th.columns.map(str)
aluminium_7th.columns = aluminium_7th.columns.map(str)
cement_7th.columns = cement_7th.columns.map(str)
chemicals_7th.columns = chemicals_7th.columns.map(str)
mining_7th.columns = mining_7th.columns.map(str)
pulp_paper_7th.columns = pulp_paper_7th.columns.map(str)
non_specified_7th.columns = non_specified_7th.columns.map(str)
other_7th.columns = other_7th.columns.map(str)

In [35]:
# Some things required to build the workbook

# Economy vector
economies = historical_production['economy'].unique()

# Year vector 1980 to 2050
years = list(range(1980, 2051))

# Make space for charts
chart_height = 18 # number of excel rows
space_for_results = 18

# Series

series = ['GDP', 'GDP growth', 'Population', 'GDP per capita', 'GDP per capita growth']
sector = ['agriculture_gdp_ppp_2017_usd', 'industry_gdp_ppp_2017_usd', 'manufacturing_gdp_ppp_2017_usd', 'services_gdp_ppp_2017_usd', 'agriculture_gdp_share',
          'industry_gdp_share', 'manufacturing_gdp_share', 'services_gdp_share']
chemicals = ['ammonia_production', 'ethylene_production', 'propylene_production', 'benzene_production', 'toluene_production', 'xylene_production']
chemicals_7 = ['ammonia_production_7th', 'ethylene_production_7th', 'propylene_production_7th', 'benzene_production_7th', 'toluene_production_7th', 'xylene_production_7th', 'Ammonia_7th', 'Urea_7th', 'Methanol_7th']
pulp_paper = ['pulp_mechanical_production', 'pulp_semi_chemical_production', 'pulp_chemical_production', 'pulp_recovered_production', 'paper_news_production', 
              'paper_writing_production', 'paper_paperboard_production', 'paper_recovered_production']
pulp_paper_7 = ['paper_paperboard_production_7th']

# Index series

steel_index = ['steel_production_pc_index', 'steel_production_7th_index', 'GDP_per_capita_index']
cement_index = ['cement_production_pc_index', 'cement_production_7th_index', 'GDP_per_capita_index']
aluminium_index = ['aluminium_production_pc_index', 'aluminium_production_7th_index', 'GDP_per_capita_index']
chemicals_index = ['chem_prod_pc_index', 'ammonia_production_7th_index', 'ethylene_production_7th_index', 'propylene_production_7th_index', 'benzene_production_7th_index',
                   'toluene_production_7th_index', 'xylene_production_7th_index', 'GDP_per_capita_index', 'Urea_7th_index', 'Ammonia_7th_index', 'Methanol_7th_index']
pp_index = ['pp_prod_pc_index', 'paper_pulp_production_7th_index', 'GDP_per_capita_index']
mining_index = ['mining_gva_pc_index', 'mining_production_7th_index', 'GDP_per_capita_index']
fb_index = ['food_beverage_tobacco_gva_pc_index', 'other_production_7th_index', 'GDP_per_capita_index']
textiles_index = ['textiles_gva_pc_index', 'other_production_7th_index', 'GDP_per_capita_index']
wood_index = ['wood_gva_pc_index', 'other_production_7th_index', 'GDP_per_capita_index']
machinery_index = ['machinery_gva_pc_index', 'other_production_7th_index', 'GDP_per_capita_index']
vehicles_index = ['vehicles_gva_pc_index', 'other_production_7th_index', 'GDP_per_capita_index']
oth_transport_index = ['other_transport_gva_pc_index', 'other_production_7th_index', 'GDP_per_capita_index']
construction_index = ['construction_gva_pc_index', 'other_production_7th_index', 'GDP_per_capita_index']

# Colours

colours = pd.read_excel('../data/colour_template_7th.xlsx')
colours_hex = colours['hex']

In [49]:
##################################### Build initial look workbook #####################################

# Define directory where to save the workbook
script_dir = '../results/'
results_dir = os.path.join(script_dir, 'initial_look/')
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

# Create a pandas ExcelWriter workbook using xlsxwriter as the engine and save it to the directory created above

writer = pd.ExcelWriter(results_dir + 'production_projection_workbook.xlsx', engine = 'xlsxwriter')
pandas.io.formats.excel.ExcelFormatter.header_style = None

# Make workbooks with requisite data

for economy in economies:
    # Empty macro dataframe and empty sectoral df
    macro_df = pd.DataFrame()
    sectoral_df = pd.DataFrame()

    # The requisite macro data to populate macro dataframe
    GDP_build = GDP[GDP['Economy'] == economy].reset_index(drop = True)
    GDP_build['Series'] = 'GDP'
    GDP_growth_build = GDP_growth[GDP_growth['Economy'] == economy].reset_index(drop = True)
    GDP_growth_build['Series'] = 'GDP growth'
    pop_build = population[population['Economy'] == economy].reset_index(drop = True)
    pop_build['Series'] = 'Population' 
    GDPpercapita_build = GDP_per_capita[GDP_per_capita['Economy'] == economy].reset_index(drop = True)
    GDPpercapita_build['Series'] = 'GDP per capita'
    GDPpercapita_pc_build = GDP_per_capita_percent[GDP_per_capita_percent['Economy'] == economy].reset_index(drop = True)
    GDPpercapita_pc_build['Series'] = 'GDP per capita growth'

    macro_df = macro_df.append([GDP_build, GDP_growth_build, pop_build, GDPpercapita_build, GDPpercapita_pc_build]).reset_index(drop = True)[['Economy', 'Series', 'Unit'] + list(range(1990, 2051, 1))]

    macro_rows = macro_df.shape[0]
    macro_cols = macro_df.shape[1]

    # Now build some aspects of the macro data frame to use in charts and tables below

    # GDP per cap indexed
    gdp_pc_index = macro_df[macro_df['Series'] == 'GDP per capita'].reset_index(drop = True).copy()
    gdp_pc_index = gdp_pc_index.select_dtypes(exclude = 'object') * 100 / gdp_pc_index.loc[0, 2017]
    gdp_pc_index['economy'] = economy
    gdp_pc_index['unit'] = 'index'
    gdp_pc_index['item'] = 'GDP_per_capita_index'
    gdp_pc_index.columns = gdp_pc_index.columns.map(str)
    
    # Sectoral df
    sectGDP_build = sectoral_GDP[sectoral_GDP['Economy'] == economy].reset_index(drop = True)
    sectGDPpc_build = sectoral_GDP_percent[sectoral_GDP_percent['Economy'] == economy].reset_index(drop = True)

    sectoral_df = sectoral_df.append([sectGDP_build, sectGDPpc_build]).reset_index(drop = True)

    sectoral_rows = sectoral_df.shape[0]
    sectoral_cols = sectoral_df.shape[1]

    # Now create the historical production dataframes

    steel_df1 = historical_production[(historical_production['item'].isin(['steel_production', 'steel_production_pc'])) &
                                      (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    steel_prod_7th = steel_7th[(steel_7th['economy'] == economy) &
                               (steel_7th['item'] == 'steel_production_7th')].copy().reset_index(drop = True)

    steel_prod_7th_index = steel_prod_7th.select_dtypes(exclude = 'object') * 100 / steel_prod_7th.loc[0, '2017']
    steel_prod_7th_index.replace([np.inf, -np.inf], np.nan, inplace = True)
    steel_prod_7th_index['economy'] = economy
    steel_prod_7th_index['unit'] = 'index'
    steel_prod_7th_index['item'] = 'steel_production_7th_index'

    steel_prod_pc1 = steel_df1[steel_df1['item'] == 'steel_production_pc'].copy().reset_index(drop = True)
    steel_prod_pc2 = steel_prod_pc1.select_dtypes(exclude = 'object') * 100 / steel_prod_pc1.loc[0, '2017']
    steel_prod_pc2['economy'] = economy
    steel_prod_pc2['unit'] = 'index'
    steel_prod_pc2['item'] = 'steel_production_pc_index' 

    steel_df1 = steel_df1.append([steel_prod_pc2, steel_prod_7th, steel_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_steel_df = {'economy': economy, 'unit': 'thousand tonnes', 'item': '8th_steel_production_projection'}
    interim_working_df = {'economy': 'Calc_row'}
    steel_df1 = steel_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_steel_df], ignore_index = True)

    steel_df1_nrows = steel_df1.shape[0]
    steel_df1_ncols = steel_df1.shape[1]

    cement_df1 = historical_production[(historical_production['item'].isin(['cement_production', 'cement_production_pc'])) &
                                       (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    cement_prod_7th = cement_7th[cement_7th['economy'] == economy].copy().reset_index(drop = True)

    cement_prod_7th_index = cement_prod_7th.select_dtypes(exclude = 'object') * 100 / cement_prod_7th.loc[0, '2017']
    cement_prod_7th_index['economy'] = economy
    cement_prod_7th_index['unit'] = 'index'
    cement_prod_7th_index['item'] = 'cement_production_7th_index'

    cement_prod_pc1 = cement_df1[cement_df1['item'] == 'cement_production_pc'].copy().reset_index(drop = True)
    cement_prod_pc2 = cement_prod_pc1.select_dtypes(exclude = 'object') * 100 / cement_prod_pc1.loc[0, '2017']
    cement_prod_pc2['economy'] = economy
    cement_prod_pc2['unit'] = 'index'
    cement_prod_pc2['item'] = 'cement_production_pc_index'

    cement_df1 = cement_df1.append([cement_prod_pc2, cement_prod_7th, cement_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_cement_df = {'economy': economy, 'unit': 'thousand tonnes', 'item': '8th_cement_production_projection'}
    cement_df1 = cement_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_cement_df], ignore_index = True)

    cement_df1_nrows = cement_df1.shape[0]
    cement_df1_ncols = cement_df1.shape[1]

    aluminium_df1 = historical_production[(historical_production['item'].isin(['aluminium_production', 'aluminium_production_pc'])) &
                                           (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    al_prod_7th = aluminium_7th[cement_7th['economy'] == economy].copy().reset_index(drop = True)

    al_prod_7th_index = al_prod_7th.select_dtypes(exclude = 'object') * 100 / al_prod_7th.loc[0, '2017']
    al_prod_7th_index['economy'] = economy
    al_prod_7th_index['unit'] = 'index'
    al_prod_7th_index['item'] = 'aluminium_production_7th_index'

    aluminium_prod_pc1 = aluminium_df1[aluminium_df1['item'] == 'aluminium_production_pc'].copy().reset_index(drop = True)
    aluminium_prod_pc2 = aluminium_prod_pc1.select_dtypes(exclude = 'object') * 100 / aluminium_prod_pc1.loc[0, '2017']
    aluminium_prod_pc2['economy'] = economy
    aluminium_prod_pc2['unit'] = 'index'
    aluminium_prod_pc2['item'] = 'aluminium_production_pc_index'

    aluminium_df1 = aluminium_df1.append([aluminium_prod_pc2, al_prod_7th, al_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_al_df = {'economy': economy, 'unit': 'thousand tonnes', 'item': '8th_aluminium_production_projection'}
    aluminium_df1 = aluminium_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_al_df], ignore_index = True)

    aluminium_df1_nrows = aluminium_df1.shape[0]
    aluminium_df1_ncols = aluminium_df1.shape[1]

    chemicals_df1 = historical_production[(historical_production['item'].isin(['ammonia_production', 'ethylene_production', 'propylene_production', 'benzene_production', 
                                                                               'toluene_production', 'xylene_production'])) &
                                          (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    chem_pc_agg = historical_production[(historical_production['item'].isin(['ammonia_production_pc', 'ethylene_production_pc', 'propylene_production_pc', 'benzene_production_pc', 
                                                                               'toluene_production_pc', 'xylene_production_pc'])) &
                                          (historical_production['economy'] == economy)].copy().groupby(['economy', 'unit']).sum().assign(item = 'chem_prod_pc', economy = economy, unit = 'tonnes per person')\
                                              .reset_index(drop = True)

    chem_pc_agg_index = chem_pc_agg.select_dtypes(exclude = 'object') * 100 / chem_pc_agg.loc[0, '2017']
    chem_pc_agg_index['economy'] = economy
    chem_pc_agg_index['unit'] = 'index'
    chem_pc_agg_index['item'] = 'chem_prod_pc_index'

    chem_prod_7th = chemicals_7th[(chemicals_7th['economy'] == economy) &
                                  (chemicals_7th['item'].isin(['ammonia_production_7th', 'ethylene_production_7th', 'propylene_production_7th', 'benzene_production_7th', 
                                                               'toluene_production_7th', 'xylene_production_7th', 'Urea_7th', 'Ammonia_7th', 'Methanol_7th']))].copy().reset_index(drop = True)

    chem_prod_7th_index = pd.DataFrame()

    for i in range(chem_prod_7th.shape[0]):
        chem_interim = chem_prod_7th.iloc[i, :].to_frame().transpose().iloc[0,4:] * 100 / chem_prod_7th.loc[i, '2017']
        chem_interim['item'] = chem_prod_7th.iloc[i, 1] + '_index'
        chem_prod_7th_index = chem_prod_7th_index.append(chem_interim)
    
    chem_prod_7th_index['economy'] = economy
    chem_prod_7th_index['unit'] = 'index'

    chemicals_df1 = chemicals_df1.append([chem_pc_agg, chem_pc_agg_index, chem_prod_7th, chem_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_chem_df = {'economy': economy, 'unit': 'thousand tonnes', 'item': '8th_chemicals_production_projection'}
    chemicals_df1 = chemicals_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_chem_df], ignore_index = True)

    chemical_rows = chemicals_df1.shape[0]
    chemical_cols = chemicals_df1.shape[1]

    pulppaper_df1 = historical_production[(historical_production['item'].isin(['pulp_mechanical_production', 'pulp_semi_chemical_production', 'pulp_chemical_production',
                                                                               'pulp_recovered_production', 'paper_news_production', 'paper_writing_production', 'paper_paperboard_production', 
                                                                               'paper_recovered_production'])) &
                                          (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    pp_pc_agg = historical_production[(historical_production['item'].isin(['pulp_mechanical_production_pc', 'pulp_semi_chemical_production_pc', 'pulp_chemical_production_pc',
                                                                               'pulp_recovered_production_pc', 'paper_news_production_pc', 'paper_writing_production_pc', 'paper_paperboard_production_pc', 
                                                                               'paper_recovered_production_pc'])) &
                                          (historical_production['economy'] == economy)].copy().groupby(['economy', 'unit']).sum().assign(item = 'pp_prod_pc', economy = economy, unit = 'tonnes per thousand persons')\
                                              .reset_index(drop = True)

    pp_pc_agg_index = pp_pc_agg.select_dtypes(exclude = 'object') * 100 / pp_pc_agg.loc[0, '2017']
    pp_pc_agg_index['economy'] = economy
    pp_pc_agg_index['unit'] = 'index'
    pp_pc_agg_index['item'] = 'pp_prod_pc_index'
    
    pp_prod_7th = pulp_paper_7th[(pulp_paper_7th['economy'] == economy) &
                                 (pulp_paper_7th['item'] == 'paper_paperboard_production_7th')].copy().reset_index(drop = True)

    pp_prod_7th_index = pp_prod_7th.select_dtypes(exclude = 'object') * 100 / pp_prod_7th.loc[0, '2017']
    pp_prod_7th_index['economy'] = economy
    pp_prod_7th_index['unit'] = 'index'
    pp_prod_7th_index['item'] = 'paper_pulp_production_7th_index'

    pulppaper_df1 = pulppaper_df1.append([pp_pc_agg, pp_pc_agg_index, pp_prod_7th, pp_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_pp_df = {'economy': economy, 'unit': 'tonnes', 'item': '8th_pulp_paper_production_projection'}
    pulppaper_df1 = pulppaper_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_pp_df], ignore_index = True)

    pulppaper_rows = pulppaper_df1.shape[0]
    pulppaper_cols = pulppaper_df1.shape[1]

    mining_df1 = historical_production[(historical_production['item'].isin(['mining_gva', 'mining_gva_pc'])) &
                                       (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    mining_prod_7th = mining_7th[mining_7th['economy'] == economy].copy().reset_index(drop = True)

    mining_prod_7th_index = mining_prod_7th.select_dtypes(exclude = 'object') * 100 / mining_prod_7th.loc[0, '2017']
    mining_prod_7th_index['economy'] = economy
    mining_prod_7th_index['unit'] = 'index'
    mining_prod_7th_index['item'] = 'mining_production_7th_index'

    mining_prod_pc1 = mining_df1[mining_df1['item'] == 'mining_gva_pc'].copy().reset_index(drop = True)
    mining_prod_pc2 = mining_prod_pc1.select_dtypes(exclude = 'object') * 100 / mining_prod_pc1.loc[0, '2017']
    mining_prod_pc2['economy'] = economy
    mining_prod_pc2['unit'] = 'index'
    mining_prod_pc2['item'] = 'mining_gva_pc_index'

    mining_df1 = mining_df1.append([mining_prod_pc2, mining_prod_7th, mining_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_mining_df = {'economy': economy, 'unit': 'GVA millions USD', 'item': '8th_mining_production_projection'}
    mining_df1 = mining_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_mining_df], ignore_index = True)

    mining_rows = mining_df1.shape[0]
    mining_cols = mining_df1.shape[1]

    fb_df1 = historical_production[(historical_production['item'].isin(['food_beverage_tobacco_gva', 'food_beverage_tobacco_gva_pc'])) &
                                   (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    other_prod_7th = other_7th[other_7th['economy'] == economy].copy().reset_index(drop = True)

    other_prod_7th_index = other_prod_7th.select_dtypes(exclude = 'object') * 100 / other_prod_7th.loc[0, '2017']
    other_prod_7th_index['economy'] = economy
    other_prod_7th_index['unit'] = 'index'
    other_prod_7th_index['item'] = 'other_production_7th_index'

    fb_prod_pc1 = fb_df1[fb_df1['item'] == 'food_beverage_tobacco_gva_pc'].copy().reset_index(drop = True)
    fb_prod_pc2 = fb_prod_pc1.select_dtypes(exclude = 'object') * 100 / fb_prod_pc1.loc[0, '2017']
    fb_prod_pc2['economy'] = economy
    fb_prod_pc2['unit'] = 'index'
    fb_prod_pc2['item'] = 'food_beverage_tobacco_gva_pc_index'

    fb_df1 = fb_df1.append([fb_prod_pc2, other_prod_7th, other_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_fb_df = {'economy': economy, 'unit': 'GVA millions USD', 'item': '8th_food_beverage_tobacco_production_projection'}
    fb_df1 = fb_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_fb_df], ignore_index = True)

    fb_rows = fb_df1.shape[0]
    fb_cols = fb_df1.shape[1]

    textiles_df1 = historical_production[(historical_production['item'].isin(['textiles_gva', 'textiles_gva_pc'])) &
                                         (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    textiles_prod_pc1 = textiles_df1[textiles_df1['item'] == 'textiles_gva_pc'].copy().reset_index(drop = True)
    textiles_prod_pc2 = textiles_prod_pc1.select_dtypes(exclude = 'object') * 100 / textiles_prod_pc1.loc[0, '2017']
    textiles_prod_pc2['economy'] = economy
    textiles_prod_pc2['unit'] = 'index'
    textiles_prod_pc2['item'] = 'textiles_gva_pc_index'

    textiles_df1 = textiles_df1.append([textiles_prod_pc2, other_prod_7th, other_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_textiles_df = {'economy': economy, 'unit': 'GVA millions USD', 'item': '8th_textiles_production_projection'}
    textiles_df1 = textiles_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_textiles_df], ignore_index = True)       

    textiles_rows = textiles_df1.shape[0]
    textiles_cols = textiles_df1.shape[1]                                         

    wood_df1 = historical_production[(historical_production['item'].isin(['wood_gva', 'wood_gva_pc'])) &
                                     (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    wood_prod_pc1 = wood_df1[wood_df1['item'] == 'wood_gva_pc'].copy().reset_index(drop = True)
    wood_prod_pc2 = wood_prod_pc1.select_dtypes(exclude = 'object') * 100 / wood_prod_pc1.loc[0, '2017']
    wood_prod_pc2['economy'] = economy
    wood_prod_pc2['unit'] = 'index'
    wood_prod_pc2['item'] = 'wood_gva_pc_index'

    wood_df1 = wood_df1.append([wood_prod_pc2, other_prod_7th, other_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_wood_df = {'economy': economy, 'unit': 'GVA millions USD', 'item': '8th_wood_production_projection'}
    wood_df1 = wood_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_wood_df], ignore_index = True)

    wood_rows = wood_df1.shape[0]
    wood_cols = wood_df1.shape[1]                                     

    machinery_df1 = historical_production[(historical_production['item'].isin(['machinery_gva', 'machinery_gva_pc'])) &
                                          (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    machinery_prod_pc1 = machinery_df1[machinery_df1['item'] == 'machinery_gva_pc'].copy().reset_index(drop = True)
    machinery_prod_pc2 = machinery_prod_pc1.select_dtypes(exclude = 'object') * 100 / machinery_prod_pc1.loc[0, '2017']
    machinery_prod_pc2['economy'] = economy
    machinery_prod_pc2['unit'] = 'index'
    machinery_prod_pc2['item'] = 'machinery_gva_pc_index'
    
    machinery_df1 = machinery_df1.append([machinery_prod_pc2, other_prod_7th, other_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_mach_df = {'economy': economy, 'unit': 'GVA millions USD', 'item': '8th_machinery_production_projection'}
    machinery_df1 = machinery_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_mach_df], ignore_index = True)

    mach_rows = machinery_df1.shape[0]
    mach_cols = machinery_df1.shape[1]

    vehicles_df1 = historical_production[(historical_production['item'].isin(['vehicles_gva', 'vehicles_gva_pc'])) &
                                         (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    vehicles_prod_pc1 = vehicles_df1[vehicles_df1['item'] == 'vehicles_gva_pc'].copy().reset_index(drop = True)
    vehicles_prod_pc2 = vehicles_prod_pc1.select_dtypes(exclude = 'object') * 100 / vehicles_prod_pc1.loc[0, '2017']
    vehicles_prod_pc2['economy'] = economy
    vehicles_prod_pc2['unit'] = 'index'
    vehicles_prod_pc2['item'] = 'vehicles_gva_pc_index'

    vehicles_df1 = vehicles_df1.append([vehicles_prod_pc2, other_prod_7th, other_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_vehicles_df = {'economy': economy, 'unit': 'GVA millions USD', 'item': '8th_vehicles_production_projection'}
    vehicles_df1 = vehicles_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_vehicles_df], ignore_index = True)

    vehicles_rows = vehicles_df1.shape[0]
    vehicles_cols = vehicles_df1.shape[1]

    oth_trans_df1 = historical_production[(historical_production['item'].isin(['other_transport_gva', 'other_transport_gva_pc'])) &
                                          (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    oth_trans_prod_pc1 = oth_trans_df1[oth_trans_df1['item'] == 'other_transport_gva_pc'].copy().reset_index(drop = True)
    oth_trans_prod_pc2 = oth_trans_prod_pc1.select_dtypes(exclude = 'object') * 100 / oth_trans_prod_pc1.loc[0, '2017']
    oth_trans_prod_pc2['economy'] = economy
    oth_trans_prod_pc2['unit'] = 'index'
    oth_trans_prod_pc2['item'] = 'other_transport_gva_pc_index'

    oth_trans_df1 = oth_trans_df1.append([oth_trans_prod_pc2, other_prod_7th, other_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_othtrans_df = {'economy': economy, 'unit': 'GVA millions USD', 'item': '8th_other_transport_production_projection'}
    oth_trans_df1 = oth_trans_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_othtrans_df], ignore_index = True)

    oth_trans_rows = oth_trans_df1.shape[0]
    oth_trans_cols = oth_trans_df1.shape[1]

    construction_df1 = historical_production[(historical_production['item'].isin(['construction_gva', 'construction_gva_pc'])) &
                                             (historical_production['economy'] == economy)].copy().reset_index(drop = True)

    construction_prod_pc1 = construction_df1[construction_df1['item'] == 'construction_gva_pc'].copy().reset_index(drop = True)
    construction_prod_pc2 = construction_prod_pc1.select_dtypes(exclude = 'object') * 100 / construction_prod_pc1.loc[0, '2017']
    construction_prod_pc2['economy'] = economy
    construction_prod_pc2['unit'] = 'index'
    construction_prod_pc2['item'] = 'construction_gva_pc_index'

    construction_df1 = construction_df1.append([construction_prod_pc2, other_prod_7th, other_prod_7th_index, gdp_pc_index]).replace([np.inf, -np.inf], np.nan).dropna(how = 'all').reset_index(drop = True)
    interim_cons_df = {'economy': economy, 'unit': 'GVA millions USD', 'item': '8th_construction_production_projection'}
    construction_df1 = construction_df1.append([interim_working_df, interim_working_df, interim_working_df, interim_cons_df], ignore_index = True)

    construction_rows = construction_df1.shape[0]
    construction_cols = construction_df1.shape[1]

    # A place to place consolidated_results

    results_df = pd.DataFrame(columns = steel_df1.columns)
    
    results_df = results_df.append([interim_steel_df, interim_cement_df, interim_al_df, interim_chem_df, interim_pp_df, interim_mining_df, interim_fb_df, interim_textiles_df, interim_wood_df,
                                    interim_mach_df, interim_vehicles_df, interim_othtrans_df, interim_cons_df], ignore_index = True)

    results_df_rows = results_df.shape[0]
    results_df_cols = results_df.shape[1]                                         
                                                                                  
    # place the dataframes within the initial look workbook
    results_df.to_excel(writer, sheet_name = economy, index = False, startrow = 4)
    macro_df.to_excel(writer, sheet_name = economy, index = False, startrow = chart_height + space_for_results)
    sectoral_df.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 2) + space_for_results + macro_rows + 1)
    steel_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 3) + space_for_results + macro_rows + sectoral_rows + 2)
    cement_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 4) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + 4)
    aluminium_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 5) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + 6)
    chemicals_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 6) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows +\
        aluminium_df1_nrows + 8)
    pulppaper_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 7) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows +\
        aluminium_df1_nrows + chemical_rows + 10)
    mining_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 8) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows +\
        chemical_rows + pulppaper_rows + 12)
    fb_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 9) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows +\
        chemical_rows + pulppaper_rows + mining_rows + 14)
    textiles_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 10) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows +\
        chemical_rows + pulppaper_rows + mining_rows + fb_rows + 16)
    wood_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 11) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows +\
        chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 18)
    machinery_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 12) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows +\
        chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 20)
    vehicles_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 13) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows +\
        chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + 22)
    oth_trans_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 14) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows +\
        chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + 24)
    construction_df1.to_excel(writer, sheet_name = economy, index = False, startrow = (chart_height * 15) + space_for_results + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows +\
        chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + oth_trans_rows + 26) 

    # Access the workbook
    workbook = writer.book
    econ_worksheet = writer.sheets[economy]

    # Comma format and header format
    comma_format = workbook.add_format({'num_format': '#,##0.00'})
    header_format = workbook.add_format({'font_name': 'Calibri', 'font_size': 11, 'bold': True})
    cell_format1 = workbook.add_format({'bold': True})
    finished_format1 = workbook.add_format({'bg_color': '#add8e6'})

    # Apply formatting
    econ_worksheet.set_column(3, steel_df1_ncols + 1, None, comma_format)
    econ_worksheet.set_row(4, None, header_format)
    econ_worksheet.set_row((chart_height * 2), None, header_format)
    econ_worksheet.set_row((chart_height * 3) + macro_rows + 1, None, header_format)
    econ_worksheet.set_row((chart_height * 4) + macro_rows + sectoral_rows + 2, None, header_format)
    econ_worksheet.set_row((chart_height * 5) + macro_rows + sectoral_rows + steel_df1_nrows + 4, None, header_format)
    econ_worksheet.set_row((chart_height * 6) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + 6, None, header_format)
    econ_worksheet.set_row((chart_height * 7) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 8, None, header_format)
    econ_worksheet.set_row((chart_height * 8) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 10, None, header_format)
    econ_worksheet.set_row((chart_height * 9) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 12, None, header_format)
    econ_worksheet.set_row((chart_height * 10) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + 14, None, header_format)
    econ_worksheet.set_row((chart_height * 11) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + 16, None, header_format)
    econ_worksheet.set_row((chart_height * 12) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + 18, None, header_format)
    econ_worksheet.set_row((chart_height * 13) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + 20, None, header_format)
    econ_worksheet.set_row((chart_height * 14) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + 22, None, header_format)
    econ_worksheet.set_row((chart_height * 15) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + 24, None, header_format)
    econ_worksheet.set_row((chart_height * 16) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + oth_trans_rows + 26, None, header_format)
    econ_worksheet.write(0, 0, economy + ' industry production projections', cell_format1)
    econ_worksheet.write(2, 0, 'Place results calculated below in this top table (from the blue rows below)', cell_format1)
    
    # Format to fill in 
    econ_worksheet.set_row((chart_height * 4) + macro_rows + sectoral_rows + steel_df1_nrows + 2, None, finished_format1)
    econ_worksheet.set_row((chart_height * 5) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + 4, None, finished_format1)
    econ_worksheet.set_row((chart_height * 6) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 6, None, finished_format1)
    econ_worksheet.set_row((chart_height * 7) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 8, None, finished_format1)
    econ_worksheet.set_row((chart_height * 8) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 10, None, finished_format1)
    econ_worksheet.set_row((chart_height * 9) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + 12, None, finished_format1)
    econ_worksheet.set_row((chart_height * 10) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + 14, None, finished_format1)
    econ_worksheet.set_row((chart_height * 11) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + 16, None, finished_format1)
    econ_worksheet.set_row((chart_height * 12) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + 18, None, finished_format1)
    econ_worksheet.set_row((chart_height * 13) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + 20, None, finished_format1)
    econ_worksheet.set_row((chart_height * 14) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + 22, None, finished_format1)
    econ_worksheet.set_row((chart_height * 15) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + oth_trans_rows + 24, None, finished_format1)
    econ_worksheet.set_row((chart_height * 16) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + oth_trans_rows + construction_rows + 26, None, finished_format1)

    # Create a GDP line chart
    macro1 = workbook.add_chart({'type': 'line'})
    macro1.set_size({
        'width': 500,
        'height': 300
    })
    
    macro1.set_chartarea({
        'border': {'none': True}
    })
    
    macro1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    macro1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'GDP',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    macro1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    macro1.set_title({
        'none': True
    })
        
    for item in series[:1]:
        i = macro_df[macro_df['Series'] == item].index[0]
        macro1.add_series({
            'name':       [economy, i + 1 + (chart_height * 2), 1],
            'categories': [economy, chart_height * 2, 3, chart_height * 2, macro_cols - 1],
            'values':     [economy, i + 1 + (chart_height * 2), 3, i + 1 + (chart_height * 2), macro_cols - 1],
            'line':       {'color': 'blue', 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('B21', macro1)

    # Create a GDP growth column chart
    macro2 = workbook.add_chart({'type': 'column'})
    macro2.set_size({
        'width': 500,
        'height': 300
    })
    
    macro2.set_chartarea({
        'border': {'none': True}
    })
    
    macro2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    macro2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'GDP growth',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    macro2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    macro2.set_title({
        'none': True
    })
        
    for item in series[1:2]:
        i = macro_df[macro_df['Series'] == item].index[0]
        macro2.add_series({
            'name':       [economy, i + 1 + (chart_height * 2), 1],
            'categories': [economy, chart_height * 2, 3, chart_height * 2, macro_cols - 1],
            'values':     [economy, i + 1 + (chart_height * 2), 3, i + 1 + (chart_height * 2), macro_cols - 1],
            'line':       {'color': 'blue', 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('J21', macro2)

    # Create a population line chart
    macro3 = workbook.add_chart({'type': 'line'})
    macro3.set_size({
        'width': 500,
        'height': 300
    })
    
    macro3.set_chartarea({
        'border': {'none': True}
    })
    
    macro3.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    macro3.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Population',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    macro3.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    macro3.set_title({
        'none': True
    })
        
    for item in series[2:3]:
        i = macro_df[macro_df['Series'] == item].index[0]
        macro3.add_series({
            'name':       [economy, i + 1 + (chart_height * 2), 1],
            'categories': [economy, chart_height * 2, 3, chart_height * 2, macro_cols - 1],
            'values':     [economy, i + 1 + (chart_height * 2), 3, i + 1 + (chart_height * 2), macro_cols - 1],
            'line':       {'color': 'blue', 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('R21', macro3)

    # Create a GDP line chart
    macro4 = workbook.add_chart({'type': 'line'})
    macro4.set_size({
        'width': 500,
        'height': 300
    })
    
    macro4.set_chartarea({
        'border': {'none': True}
    })
    
    macro4.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    macro4.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'GDP per capita',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    macro4.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    macro4.set_title({
        'none': True
    })
        
    for item in series[3:4]:
        i = macro_df[macro_df['Series'] == item].index[0]
        macro4.add_series({
            'name':       [economy, i + 1 + (chart_height * 2), 1],
            'categories': [economy, chart_height * 2, 3, chart_height * 2, macro_cols - 1],
            'values':     [economy, i + 1 + (chart_height * 2), 3, i + 1 + (chart_height * 2), macro_cols - 1],
            'line':       {'color': 'blue', 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('Z21', macro4)

    # Create a GDP growth column chart
    macro5 = workbook.add_chart({'type': 'column'})
    macro5.set_size({
        'width': 500,
        'height': 300
    })
    
    macro5.set_chartarea({
        'border': {'none': True}
    })
    
    macro5.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    macro5.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'GDP per capita growth',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    macro5.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        'none': True
    })
        
    macro5.set_title({
        'none': True
    })
        
    for item in series[4:]:
        i = macro_df[macro_df['Series'] == item].index[0]
        macro5.add_series({
            'name':       [economy, i + 1 + (chart_height * 2), 1],
            'categories': [economy, chart_height * 2, 3, chart_height * 2, macro_cols - 1],
            'values':     [economy, i + 1 + (chart_height * 2), 3, i + 1 + (chart_height * 2), macro_cols - 1],
            'line':       {'color': 'blue', 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('AH21', macro5)

    ###########################################################################################################################

    # Create a sectoral line chart
    sectoral1 = workbook.add_chart({'type': 'line'})
    sectoral1.set_size({
        'width': 700,
        'height': 300
    })
    
    sectoral1.set_chartarea({
        'border': {'none': True}
    })
    
    sectoral1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    sectoral1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Sectoral GDP',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    sectoral1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10}
        #'none': True
    })
        
    sectoral1.set_title({
        'none': True
    })
        
    for item in sector[4:8]:
        i = sectoral_df[sectoral_df['Sector'] == item].index[0]
        sectoral1.add_series({
            'name':       [economy, i + 7 + (chart_height * 3), 2],
            'categories': [economy, (chart_height * 3) + 6, 3, (chart_height * 3) + 6, sectoral_cols - 1],
            'values':     [economy, i + 7 + (chart_height * 3), 3, i + 7 + (chart_height * 3), sectoral_cols - 1],
            'line':       {'color': colours_hex[i], 'width': 1.25} 
        })
    
    econ_worksheet.insert_chart('B45', sectoral1) 

    #################################################################################################################################

    # Create a steel line chart
    steel1 = workbook.add_chart({'type': 'line'})
    steel1.set_size({
        'width': 700,
        'height': 300
    })
    
    steel1.set_chartarea({
        'border': {'none': True}
    })
    
    steel1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    steel1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Steel production (thousand tonnes)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    steel1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    steel1.set_title({
        'none': True
    })
        
    for prod_item in ['steel_production', 'steel_production_7th', '8th_steel_production_projection']:
        i = steel_df1[steel_df1['item'] == prod_item].index[0]
        steel1.add_series({
            'name':       [economy, (chart_height * 4) + 16 + i, 1],
            'categories': [economy, (chart_height * 4) + 15, 4, (chart_height * 4) + 15, steel_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 4) + 16 + i, 4, (chart_height * 4) + 16 + i, steel_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 
                           'width': 1.25} 
        })
        
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 3) + macro_rows + sectoral_rows + 2), steel1)

    #### Steel index

    # Create a steel line chart
    steel2 = workbook.add_chart({'type': 'line'})
    steel2.set_size({
        'width': 700,
        'height': 300
    })
    
    steel2.set_chartarea({
        'border': {'none': True}
    })
    
    steel2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    steel2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'index',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    steel2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    steel2.set_title({
        'none': True
    })
        
    for prod_item in steel_index:
        i = steel_df1[steel_df1['item'] == prod_item].index[0]
        steel2.add_series({
            'name':       [economy, (chart_height * 4) + 16 + i, 1],
            'categories': [economy, (chart_height * 4) + 15, 4, (chart_height * 4) + 15, steel_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 4) + 16 + i, 4, (chart_height * 4) + 16 + i, steel_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 
                           'width': 1.25} 
        })
        
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 3) + macro_rows + sectoral_rows + 2), steel2)

    #################################################################################################################################

    # Create a cement line chart
    cement1 = workbook.add_chart({'type': 'line'})
    cement1.set_size({
        'width': 700,
        'height': 300
    })
    
    cement1.set_chartarea({
        'border': {'none': True}
    })
    
    cement1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    cement1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Cement production (thousand tonnes)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    cement1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    cement1.set_title({
        'none': True
    })

    for prod_item in ['cement_production', 'cement_production_7th', '8th_cement_production_projection']:
        i = cement_df1[cement_df1['item'] == prod_item].index[0]    
        cement1.add_series({
            'name':       [economy, (chart_height * 5) + steel_df1_nrows + 18 + i, 1],
            'categories': [economy, (chart_height * 5) + steel_df1_nrows + 17, 4, (chart_height * 5) + steel_df1_nrows + 17, cement_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 5) + steel_df1_nrows + 18 + i, 4, (chart_height * 5) + steel_df1_nrows + 18 + i, cement_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 
                           'width': 1.25}
                           })
    
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 4) + macro_rows + sectoral_rows + steel_df1_nrows + 2), cement1)

    ####### Cement index chart

    # Create a cement line chart
    cement2 = workbook.add_chart({'type': 'line'})
    cement2.set_size({
        'width': 700,
        'height': 300
    })
    
    cement2.set_chartarea({
        'border': {'none': True}
    })
    
    cement2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    cement2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    cement2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    cement2.set_title({
        'none': True
    })

    for prod_item in cement_index:
        i = cement_df1[cement_df1['item'] == prod_item].index[0]    
        cement2.add_series({
            'name':       [economy, (chart_height * 5) + steel_df1_nrows + 18 + i, 1],
            'categories': [economy, (chart_height * 5) + steel_df1_nrows + 17, 4, (chart_height * 5) + steel_df1_nrows + 17, cement_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 5) + steel_df1_nrows + 18 + i, 4, (chart_height * 5) + steel_df1_nrows + 18 + i, cement_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 
                           'width': 1.25}
                           })
    
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 4) + macro_rows + sectoral_rows + steel_df1_nrows + 2), cement2)

    #################################################################################################################################

    # Create an aluminium line chart
    alum1 = workbook.add_chart({'type': 'line'})
    alum1.set_size({
        'width': 700,
        'height': 300
    })
    
    alum1.set_chartarea({
        'border': {'none': True}
    })
    
    alum1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    alum1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Aluminium production (thousand tonnes)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    alum1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    alum1.set_title({
        'none': True
    })

    for prod_item in ['aluminium_production', 'aluminium_production_7th', '8th_aluminium_production_projection']:
        i = aluminium_df1[aluminium_df1['item'] == prod_item].index[0]
        alum1.add_series({
            'name':       [economy, (chart_height * 6) + steel_df1_nrows + cement_df1_nrows + 20 + i, 1],
            'categories': [economy, (chart_height * 6) + steel_df1_nrows + cement_df1_nrows + 19, 4, (chart_height * 6) + steel_df1_nrows + cement_df1_nrows + 19, aluminium_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 6) + steel_df1_nrows + cement_df1_nrows + 20 + i, 4, (chart_height * 6) + steel_df1_nrows + cement_df1_nrows + 20 + i, aluminium_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 5) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + 2 + (1 * 1)), alum1)

    ############## Aluminium index chart

    # Create an aluminium line chart
    alum2 = workbook.add_chart({'type': 'line'})
    alum2.set_size({
        'width': 700,
        'height': 300
    })
    
    alum2.set_chartarea({
        'border': {'none': True}
    })
    
    alum2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    alum2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    alum2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    alum2.set_title({
        'none': True
    })

    for prod_item in aluminium_index:
        i = aluminium_df1[aluminium_df1['item'] == prod_item].index[0]
        alum2.add_series({
            'name':       [economy, (chart_height * 6) + steel_df1_nrows + cement_df1_nrows + 20 + i, 1],
            'categories': [economy, (chart_height * 6) + steel_df1_nrows + cement_df1_nrows + 19, 4, (chart_height * 6) + steel_df1_nrows + cement_df1_nrows + 19, aluminium_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 6) + steel_df1_nrows + cement_df1_nrows + 20 + i, 4, (chart_height * 6) + steel_df1_nrows + cement_df1_nrows + 20 + i, aluminium_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 5) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + 2 + (1 * 1)), alum2)

    ############################################################################################################################

    # Create a chemocals production line chart
    chem1 = workbook.add_chart({'type': 'line'})
    chem1.set_size({
        'width': 700,
        'height': 300
    })
    
    chem1.set_chartarea({
        'border': {'none': True}
    })
    
    chem1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    chem1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Chemicals production (thousand tonnes)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    chem1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10}
        #'none': True
    })
        
    chem1.set_title({
        'none': True
    })
        
    for item in (chemicals + chemicals_7):
        if chemicals_df1[chemicals_df1['item'] == item].shape[0] > 0:
            i = chemicals_df1[chemicals_df1['item'] == item].index[0]
            chem1.add_series({
                'name':       [economy, (chart_height * 7) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 22 + i, 1],
                'categories': [economy, (chart_height * 7) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 21, 4,\
                    (chart_height * 7) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 21, chemical_cols - 1],
                'values':     [economy, (chart_height * 7) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 22 + i, 4,\
                    (chart_height * 7) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 22 + i, chemical_cols - 1],
                'line':       {'color': colours_hex[i], 'width': 1} 
            })
        else:
            pass
    
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 6) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 2 + (2 * 1)), chem1)

    # Create a chemicals index line chart
    chem2 = workbook.add_chart({'type': 'line'})
    chem2.set_size({
        'width': 700,
        'height': 300
    })
    
    chem2.set_chartarea({
        'border': {'none': True}
    })
    
    chem2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    chem2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    chem2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10}
        #'none': True
    })
        
    chem2.set_title({
        'none': True
    })
        
    for item in chemicals_index:
        if chemicals_df1[chemicals_df1['item'] == item].shape[0] > 0:
            i = chemicals_df1[chemicals_df1['item'] == item].index[0]
            chem2.add_series({
                'name':       [economy, (chart_height * 7) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 22 + i, 1],
                'categories': [economy, (chart_height * 7) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 21, 4,\
                    (chart_height * 7) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 21, chemical_cols - 1],
                'values':     [economy, (chart_height * 7) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 22 + i, 4,\
                    (chart_height * 7) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 22 + i, chemical_cols - 1],
                'line':       {'color': colours_hex[i - 7], 'width': 1} 
            })
        else:
            pass
    
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 6) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + 2 + (2 * 1)), chem2)

    ############################################################################################################################

    # Create a pulp paper production line chart
    pulppaper1 = workbook.add_chart({'type': 'line'})
    pulppaper1.set_size({
        'width': 700,
        'height': 300
    })
    
    pulppaper1.set_chartarea({
        'border': {'none': True}
    })
    
    pulppaper1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    pulppaper1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Pulp paper production (tonnes)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    pulppaper1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10}
        #'none': True
    })
        
    pulppaper1.set_title({
        'none': True
    })
        
    for item in (pulp_paper + pulp_paper_7):
        i = pulppaper_df1[pulppaper_df1['item'] == item].index[0]
        pulppaper1.add_series({
            'name':       [economy, (chart_height * 8) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 24 + i, 1],
            'categories': [economy, (chart_height * 8) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 23, 4,\
                (chart_height * 8) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 23, pulppaper_cols - 1],
            'values':     [economy, (chart_height * 8) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 24 + i, 4,\
                (chart_height * 8) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 24 + i, pulppaper_cols - 1],
            'line':       {'color': colours_hex[i], 'width': 1} 
        })
    
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 7) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 2 + (3 * 1)), pulppaper1)

    #################################################################################################################################

    # Create a pulp paper production line chart
    pulppaper2 = workbook.add_chart({'type': 'line'})
    pulppaper2.set_size({
        'width': 700,
        'height': 300
    })
    
    pulppaper2.set_chartarea({
        'border': {'none': True}
    })
    
    pulppaper2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    pulppaper2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    pulppaper2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10}
        #'none': True
    })
        
    pulppaper2.set_title({
        'none': True
    })
        
    for item in pp_index:
        i = pulppaper_df1[pulppaper_df1['item'] == item].index[0]
        pulppaper2.add_series({
            'name':       [economy, (chart_height * 8) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 24 + i, 1],
            'categories': [economy, (chart_height * 8) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 23, 4,\
                (chart_height * 8) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 23, pulppaper_cols - 1],
            'values':     [economy, (chart_height * 8) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 24 + i, 4,\
                (chart_height * 8) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 24 + i, pulppaper_cols - 1],
            'line':       {'color': colours_hex[i - 4], 'width': 1} 
        })
    
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 7) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + 2 + (3 * 1)), pulppaper2)

    #################################################################################################################################

    # Create an mining line chart
    mining1 = workbook.add_chart({'type': 'line'})
    mining1.set_size({
        'width': 700,
        'height': 300
    })
    
    mining1.set_chartarea({
        'border': {'none': True}
    })
    
    mining1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    mining1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Mining GVA millions',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    mining1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    mining1.set_title({
        'none': True
    })
        
    for prod_item in ['mining_gva', 'mining_gva_7th', '8th_mining_production_projection']:
        i = mining_df1[mining_df1['item'] == prod_item].index[0]
        mining1.add_series({
            'name':       [economy, (chart_height * 9) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 26 + i, 1],
            'categories': [economy, (chart_height * 9) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 25, 4,\
                (chart_height * 9) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 25, mining_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 9) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 26 + i, 4,\
                (chart_height * 9) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 26 + i, mining_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })   
    
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 8) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 2 + (4 * 1)), mining1)

    #################################################################################################################################

    # Create an mining index line chart
    mining2 = workbook.add_chart({'type': 'line'})
    mining2.set_size({
        'width': 700,
        'height': 300
    })
    
    mining2.set_chartarea({
        'border': {'none': True}
    })
    
    mining2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    mining2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    mining2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    mining2.set_title({
        'none': True
    })
        
    for prod_item in mining_index:
        i = mining_df1[mining_df1['item'] == prod_item].index[0]
        mining2.add_series({
            'name':       [economy, (chart_height * 9) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 26 + i, 1],
            'categories': [economy, (chart_height * 9) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 25, 4,\
                (chart_height * 9) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 25, mining_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 9) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 26 + i, 4,\
                (chart_height * 9) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 26 + i, mining_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 1], 'width': 1.25} 
    })   
    
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 8) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + 2 + (4 * 1)), mining2)

    #################################################################################################################################

    # Create an food and bev gva line chart
    foodbev1 = workbook.add_chart({'type': 'line'})
    foodbev1.set_size({
        'width': 700,
        'height': 300
    })
    
    foodbev1.set_chartarea({
        'border': {'none': True}
    })
    
    foodbev1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    foodbev1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Food and beverage GVA millions',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    foodbev1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    foodbev1.set_title({
        'none': True
    })

    for prod_item in ['food_beverage_tobacco_gva', 'other_gva_7th', '8th_food_beverage_tobacco_production_projection']:
        i = fb_df1[fb_df1['item'] == prod_item].index[0]
        foodbev1.add_series({
            'name':       [economy, (chart_height * 10) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + 28 + i, 1],
            'categories': [economy, (chart_height * 10) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + 27, 4,\
                (chart_height * 10) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + 27, fb_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 10) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + 28 + i, 4,\
                (chart_height * 10) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + 28 + i, fb_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
        })
        
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 9) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + 2 + (5 * 1)), foodbev1)

    #################################################################################################################################

    # Create an food and bev index line chart
    foodbev2 = workbook.add_chart({'type': 'line'})
    foodbev2.set_size({
        'width': 700,
        'height': 300
    })
    
    foodbev2.set_chartarea({
        'border': {'none': True}
    })
    
    foodbev2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    foodbev2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    foodbev2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    foodbev2.set_title({
        'none': True
    })

    for prod_item in fb_index:
        i = fb_df1[fb_df1['item'] == prod_item].index[0]
        foodbev2.add_series({
            'name':       [economy, (chart_height * 10) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + 28 + i, 1],
            'categories': [economy, (chart_height * 10) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + 27, 4,\
                (chart_height * 10) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + 27, fb_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 10) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + 28 + i, 4,\
                (chart_height * 10) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + 28 + i, fb_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
        })
        
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 9) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + 2 + (5 * 1)), foodbev2)

    #################################################################################################################################

    # Create an textiles gva line chart
    textiles1 = workbook.add_chart({'type': 'line'})
    textiles1.set_size({
        'width': 700,
        'height': 300
    })
    
    textiles1.set_chartarea({
        'border': {'none': True}
    })
    
    textiles1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    textiles1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Textiles GVA millions',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    textiles1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    textiles1.set_title({
        'none': True
    })

    for prod_item in ['textiles_gva', 'other_gva_7th', '8th_textiles_production_projection']:
        i = textiles_df1[textiles_df1['item'] == prod_item].index[0]
        textiles1.add_series({
            'name':       [economy, (chart_height * 11) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + 30 + i, 1],
            'categories': [economy, (chart_height * 11) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + 29, 4,\
                (chart_height * 11) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + 29, textiles_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 11) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + 30 + i, 4,\
                (chart_height * 11) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + 30 + i, textiles_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 10) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + 2 + (6 * 1)), textiles1)

    #################################################################################################################################

    # Create an textiles index line chart
    textiles2 = workbook.add_chart({'type': 'line'})
    textiles2.set_size({
        'width': 700,
        'height': 300
    })
    
    textiles2.set_chartarea({
        'border': {'none': True}
    })
    
    textiles2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    textiles2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    textiles2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    textiles2.set_title({
        'none': True
    })

    for prod_item in textiles_index:
        i = textiles_df1[textiles_df1['item'] == prod_item].index[0]
        textiles2.add_series({
            'name':       [economy, (chart_height * 11) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + 30 + i, 1],
            'categories': [economy, (chart_height * 11) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + 29, 4,\
                (chart_height * 11) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + 29, textiles_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 11) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + 30 + i, 4,\
                (chart_height * 11) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + 30 + i, textiles_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 10) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + 2 + (6 * 1)), textiles2)

    #################################################################################################################################

    # Create an wood and wood products gva line chart
    wood1 = workbook.add_chart({'type': 'line'})
    wood1.set_size({
        'width': 700,
        'height': 300
    })
    
    wood1.set_chartarea({
        'border': {'none': True}
    })
    
    wood1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    wood1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Wood and wood products GVA millions',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    wood1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    wood1.set_title({
        'none': True
    })

    for prod_item in ['wood_gva', 'other_gva_7th', '8th_wood_production_projection']:
        i = wood_df1[wood_df1['item'] == prod_item].index[0]
        wood1.add_series({
            'name':       [economy, (chart_height * 12) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 32 + i, 1],
            'categories': [economy, (chart_height * 12) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 31, 4,\
                (chart_height * 12) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 31, wood_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 12) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 32 + i, 4,\
                (chart_height * 12) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 32 + i, wood_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 11) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + 2 + (7 * 1)), wood1)

    #################################################################################################################################

    # Create an wood and wood products gva line chart
    wood2 = workbook.add_chart({'type': 'line'})
    wood2.set_size({
        'width': 700,
        'height': 300
    })
    
    wood2.set_chartarea({
        'border': {'none': True}
    })
    
    wood2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    wood2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 =100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    wood2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    wood2.set_title({
        'none': True
    })

    for prod_item in wood_index:
        i = wood_df1[wood_df1['item'] == prod_item].index[0]
        wood2.add_series({
            'name':       [economy, (chart_height * 12) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 32 + i, 1],
            'categories': [economy, (chart_height * 12) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 31, 4,\
                (chart_height * 12) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 31, wood_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 12) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 32 + i, 4,\
                (chart_height * 12) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + 32 + i, wood_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 11) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + 2 + (7 * 1)), wood2)

    #################################################################################################################################

    # Create an machinery line chart
    machinery1 = workbook.add_chart({'type': 'line'})
    machinery1.set_size({
        'width': 700,
        'height': 300
    })
    
    machinery1.set_chartarea({
        'border': {'none': True}
    })
    
    machinery1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    machinery1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Machinery GVA millions',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    machinery1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    machinery1.set_title({
        'none': True
    })

    for prod_item in ['machinery_gva', 'other_gva_7th', '8th_machinery_production_projection']:
        i = machinery_df1[machinery_df1['item'] == prod_item].index[0]    
        machinery1.add_series({
            'name':       [economy, (chart_height * 13) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 34 + i, 1],
            'categories': [economy, (chart_height * 13) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 33, 4,\
                (chart_height * 13) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 33, machinery_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 13) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 34 + i, 4,\
                (chart_height * 13) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 34 + i, machinery_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 12) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + 2 + (8 * 1)), machinery1)

    #################################################################################################################################

    # Create an machinery line chart
    machinery2 = workbook.add_chart({'type': 'line'})
    machinery2.set_size({
        'width': 700,
        'height': 300
    })
    
    machinery2.set_chartarea({
        'border': {'none': True}
    })
    
    machinery2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    machinery2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    machinery2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    machinery2.set_title({
        'none': True
    })

    for prod_item in machinery_index:
        i = machinery_df1[machinery_df1['item'] == prod_item].index[0]    
        machinery2.add_series({
            'name':       [economy, (chart_height * 13) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 34 + i, 1],
            'categories': [economy, (chart_height * 13) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 33, 4,\
                (chart_height * 13) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 33, machinery_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 13) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 34 + i, 4,\
                (chart_height * 13) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + 34 + i, machinery_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 12) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + 2 + (8 * 1)), machinery2)

    #################################################################################################################################

    # Create an vehicles gva line chart
    vehicles1 = workbook.add_chart({'type': 'line'})
    vehicles1.set_size({
        'width': 700,
        'height': 300
    })
    
    vehicles1.set_chartarea({
        'border': {'none': True}
    })
    
    vehicles1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    vehicles1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Vehicles GVA millions',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    vehicles1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    vehicles1.set_title({
        'none': True
    })
        
    for prod_item in ['vehicles_gva', 'other_gva_7th', '8th_vehicles_production_projection']:
        i = vehicles_df1[vehicles_df1['item'] == prod_item].index[0]
        vehicles1.add_series({
            'name':       [economy, (chart_height * 14) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows +\
                mach_rows + 36 + i, 1],
            'categories': [economy, (chart_height * 14) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows +\
                mach_rows + 35, 4, (chart_height * 14) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows +\
                    mach_rows + 35, vehicles_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 14) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows +\
                mach_rows + 36 + i, 4,(chart_height * 14) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows +\
                    mach_rows + 36 + i, vehicles_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
        })
        
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 13) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + 2 + (9 * 1)), vehicles1)

    #################################################################################################################################

    # Create an vehicles gva line chart
    vehicles2 = workbook.add_chart({'type': 'line'})
    vehicles2.set_size({
        'width': 700,
        'height': 300
    })
    
    vehicles2.set_chartarea({
        'border': {'none': True}
    })
    
    vehicles2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    vehicles2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    vehicles2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    vehicles2.set_title({
        'none': True
    })
        
    for prod_item in vehicles_index:
        i = vehicles_df1[vehicles_df1['item'] == prod_item].index[0]
        vehicles2.add_series({
            'name':       [economy, (chart_height * 14) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows +\
                mach_rows + 36 + i, 1],
            'categories': [economy, (chart_height * 14) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows +\
                mach_rows + 35, 4, (chart_height * 14) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows +\
                    mach_rows + 35, vehicles_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 14) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows +\
                mach_rows + 36 + i, 4,(chart_height * 14) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows +\
                    mach_rows + 36 + i, vehicles_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
        })
        
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 13) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + 2 + (9 * 1)), vehicles2)

    #################################################################################################################################

    # Create an other transport gva line chart
    oth_trans1 = workbook.add_chart({'type': 'line'})
    oth_trans1.set_size({
        'width': 700,
        'height': 300
    })
    
    oth_trans1.set_chartarea({
        'border': {'none': True}
    })
    
    oth_trans1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    oth_trans1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Other transport GVA millions',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    oth_trans1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    oth_trans1.set_title({
        'none': True
    })
        
    for prod_item in ['other_transport_gva', 'other_gva_7th', '8th_other_transport_production_projection']:
        i = oth_trans_df1[oth_trans_df1['item'] == prod_item].index[0]
        oth_trans1.add_series({
            'name':       [economy, (chart_height * 15) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + 38 + i, 1],
            'categories': [economy, (chart_height * 15) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + 37, 4,\
                    (chart_height * 15) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                        vehicles_rows + 37, oth_trans_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 15) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + 38 + i, 4,\
                (chart_height * 15) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + 38 + i, oth_trans_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
        })
        
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 14) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + 2 + (10 * 1)), oth_trans1)

    #################################################################################################################################

    # Create an other transport gva line chart
    oth_trans2 = workbook.add_chart({'type': 'line'})
    oth_trans2.set_size({
        'width': 700,
        'height': 300
    })
    
    oth_trans2.set_chartarea({
        'border': {'none': True}
    })
    
    oth_trans2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    oth_trans2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    oth_trans2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    oth_trans2.set_title({
        'none': True
    })
        
    for prod_item in oth_transport_index:
        i = oth_trans_df1[oth_trans_df1['item'] == prod_item].index[0]
        oth_trans2.add_series({
            'name':       [economy, (chart_height * 15) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + 38 + i, 1],
            'categories': [economy, (chart_height * 15) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + 37, 4,\
                    (chart_height * 15) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                        vehicles_rows + 37, oth_trans_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 15) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + 38 + i, 4,\
                (chart_height * 15) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + 38 + i, oth_trans_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
        })
        
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 14) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + 2 + (10 * 1)), oth_trans2)

    #################################################################################################################################

    # Create an other transport gva line chart
    construction1 = workbook.add_chart({'type': 'line'})
    construction1.set_size({
        'width': 700,
        'height': 300
    })
    
    construction1.set_chartarea({
        'border': {'none': True}
    })
    
    construction1.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    construction1.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Construction GVA millions',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    construction1.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    construction1.set_title({
        'none': True
    })
        
    for prod_item in ['construction_gva', 'other_gva_7th', '8th_construction_production_projection']:
        i = construction_df1[construction_df1['item'] == prod_item].index[0]
        construction1.add_series({
            'name':       [economy, (chart_height * 16) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + oth_trans_rows + 40 + i, 1],
            'categories': [economy, (chart_height * 16) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + oth_trans_rows + 39, 4,\
                (chart_height * 16) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + oth_trans_rows + 39, construction_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 16) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + oth_trans_rows + 40 + i, 4,\
                (chart_height * 16) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + oth_trans_rows + 40 + i, construction_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('B' + str(((chart_height + 1) * 15) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + oth_trans_rows + 2 + (11 * 1)), construction1)

    #########################################################################################

    # Create an other transport gva line chart
    construction2 = workbook.add_chart({'type': 'line'})
    construction2.set_size({
        'width': 700,
        'height': 300
    })
    
    construction2.set_chartarea({
        'border': {'none': True}
    })
    
    construction2.set_x_axis({
        'name': 'Year',
        'label_position': 'low',
        'major_tick_mark': 'none',
        'minor_tick_mark': 'none',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232', 'rotation': -45},
        'position_axis': 'on_tick',
        'interval_unit': 4,
        'line': {'color': '#bebebe'}
    })
        
    construction2.set_y_axis({
        'major_tick_mark': 'none', 
        'minor_tick_mark': 'none',
        'name': 'Index (2017 = 100)',
        'num_font': {'font': 'Segoe UI', 'size': 10, 'color': '#323232'},
        'major_gridlines': {
            'visible': True,
            'line': {'color': '#bebebe'}
        },
        'line': {'color': '#bebebe'}
    })
        
    construction2.set_legend({
        'font': {'font': 'Segoe UI', 'size': 10},
        #'none': True
    })
        
    construction2.set_title({
        'none': True
    })
        
    for prod_item in construction_index:
        i = construction_df1[construction_df1['item'] == prod_item].index[0]
        construction2.add_series({
            'name':       [economy, (chart_height * 16) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + oth_trans_rows + 40 + i, 1],
            'categories': [economy, (chart_height * 16) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + oth_trans_rows + 39, 4,\
                (chart_height * 16) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + oth_trans_rows + 39, construction_df1.shape[1] - 1],
            'values':     [economy, (chart_height * 16) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + oth_trans_rows + 40 + i, 4,\
                (chart_height * 16) + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows + mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows +\
                vehicles_rows + oth_trans_rows + 40 + i, construction_df1.shape[1] - 1],
            'line':       {'color': colours_hex[i + 2], 'width': 1.25} 
    })
        
    econ_worksheet.insert_chart('M' + str(((chart_height + 1) * 15) + macro_rows + sectoral_rows + steel_df1_nrows + cement_df1_nrows + aluminium_df1_nrows + chemical_rows + pulppaper_rows +\
        mining_rows + fb_rows + textiles_rows + wood_rows + mach_rows + vehicles_rows + oth_trans_rows + 2 + (11 * 1)), construction2)

writer.save()

In [31]:
results_df

,economy,item,unit,item_code_new,1980,1981,1982,1983,1984,1985,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,21_VN,8th_steel_production_projection,thousand tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21_VN,8th_cement_production_projection,thousand tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21_VN,8th_aluminium_production_projection,thousand tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21_VN,8th_chemicals_production_projection,thousand tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21_VN,8th_pulp_paper_production_projection,tonnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,21_VN,8th_mining_production_projection,GVA millions USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,21_VN,8th_food_beverage_tobacco_production_projection,GVA millions USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,21_VN,8th_textiles_production_projection,GVA millions USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,21_VN,8th_wood_production_projection,GVA millions USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,21_VN,8th_machinery_production_projection,GVA millions USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
